In [1]:
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta

In [17]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(24)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [12]:
dates[0]

'0701_1957'

In [21]:
class Experiment():
    def __init__(self,plate):
        self.plate = plate
    def load(self,dates,local=False):
        self.dates=dates
        if local:
            paths=[f'Data/graph_{date}_{plate}_full_labeled.csv' for date in dates]
            paths_connection = [f'Data/graph_{date}_{plate}_connection.csv' for date in dates[:-1]]
        else:
            paths = [get_path(date,self.plate,True,extension='_full_labeled.csv') for date in dates]
            paths_connection = [get_path(date,plate,True,extension='_connection.csv') for date in dates[:-1]]
        nx_graph_poss=[]
        for path in paths:
            nx_graph_poss.append(generate_nx_graph(pd.read_csv(path,
                                converters={'origin_pos' : transform_list,'end_pos' : transform_list,
                                            'pixel_list' : ast.literal_eval}),labeled=True))
        from_tip_growth_pattern=[]
        for path in paths_connection:
            from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(path)))
        nx_graphs=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
        poss = [nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
        nx_graph_clean=[]
        for graph in nx_graphs:
            S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            len_connected=[len(nx_graph.nodes) for nx_graph in S]
            nx_graph_clean.append(S[np.argmax(len_connected)])
        connections = [c[0] for c in from_tip_growth_pattern]
        growth_patterns = [c[1] for c in from_tip_growth_pattern]
        self.positions=poss
        self.nx_graph=nx_graph_clean
        self.connections = connections
        self.growth_patterns = growth_patterns
    def save(self,path=f'Data/'):
        print('wow')
        from_tip_growth_pattern_tab=[]
        for date in self.dates[:-1]:
            from_tip_growth_pattern_tab.append(pd.read_csv(get_path(date,plate,True,extension='_connection.csv')))
        for i, date in enumerate(self.dates[:-1]):
            from_tip_growth_pattern_tab[i].to_csv(path+f'connection_{date}_{plate}.csv')
        tabs_labeled=[]
        for date in self.dates:
            tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                        converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))
        for i, date in enumerate(self.dates):
            tabs_labeled[i].to_csv(path+f'graph_{date}_{plate}_full_labeled.csv')
    
            
        

In [22]:
experiment = Experiment(13)

In [ ]:
experiment.load(dates)

In [16]:
experiment.positions[0][0]

array([9248, 6724])

In [ ]:
experiment.save()